<a href="https://colab.research.google.com/github/sandipanbasu/aiml-capstone/blob/master/mrc_Evaluations.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Imports

In [1]:
import warnings
from tqdm import tqdm
import tensorflow as tf
tf.debugging.set_log_device_placement(False)
import pickle
from tensorflow.keras import layers
from tensorflow.keras import preprocessing
import numpy as np
import pandas as pd
import json
from sklearn.model_selection import train_test_split
import pprint
from tensorflow.keras.layers import Bidirectional,LSTM,Dense,Dropout,BatchNormalization,Flatten,Input
from tensorflow.keras.models import Model
from tensorflow.keras.layers import concatenate
from numpy import array
import nltk
import re
from nltk.tokenize import RegexpTokenizer
from nltk.stem import WordNetLemmatizer,PorterStemmer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize 
from tensorflow.keras.models import load_model
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

# Load Google Bucket as drive

In [2]:
from google.colab import auth
auth.authenticate_user()

project_id = 'ai-ml-capstone'
!gcloud config set project {project_id}
!gsutil ls


Updated property [core/project].
gs://aiml-capstone/


In [3]:
!echo "deb http://packages.cloud.google.com/apt gcsfuse-bionic main" > /etc/apt/sources.list.d/gcsfuse.list
!curl https://packages.cloud.google.com/apt/doc/apt-key.gpg | apt-key add -
!apt -qq update
!apt -qq install gcsfuse

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   653  100   653    0     0  25115      0 --:--:-- --:--:-- --:--:-- 25115
OK
49 packages can be upgraded. Run 'apt list --upgradable' to see them.
The following package was automatically installed and is no longer required:
  libnvidia-common-440
Use 'apt autoremove' to remove it.
The following NEW packages will be installed:
  gcsfuse
0 upgraded, 1 newly installed, 0 to remove and 49 not upgraded.
Need to get 4,274 kB of archives.
After this operation, 12.8 MB of additional disk space will be used.
Selecting previously unselected package gcsfuse.
(Reading database ... 144328 files and directories currently installed.)
Preparing to unpack .../gcsfuse_0.28.1_amd64.deb ...
Unpacking gcsfuse (0.28.1) ...
Setting up gcsfuse (0.28.1) ...


In [4]:
!mkdir gbucket
!gcsfuse --implicit-dirs aiml-capstone gbucket 
# !umount gbucket

Using mount point: /content/gbucket
Opening GCS connection...
Opening bucket...
Mounting file system...
File system has been successfully mounted.


In [3]:
!ls gbucket/lstmbaseline-0/

 context_withoutstopwords_model_epoch_24.h5
 context_withoutstopwords_model_epoch_25_lstmbaseline0_glove_nomask_gpu.h5
 full_context_withoutstopwords_model_epoch.h5
 full_context_withoutstopwords_model_epoch_lstmbaseline0_glove_nomask_gpu.h5
'NN Network.png'
 params.json
 results.csv
 tf-serve
 tokenizer.pkl
 whole_model_epoch_24.h5


# List of Models

As part of our capstone, we are in process of evaluating the following models

Data | Model | On GPU | Masking | Padding | Epoch | Location | 
--- | --- | --- | --- | --- | --- | --- | 
Without stopwords | SVM | No | - | - | - | [here](https://storage.cloud.google.com/aiml-capstone/svm/)
Without stopwords | LSTM Baseline | No | No | Pre | 25 | [here](https://storage.cloud.google.com/aiml-capstone/lstmbaseline-0/full_context_withoutstopwords_model_epoch_lstmbaseline0_nomask_gpu.h5)
Without stopwords | Deep LSTM + GloVe | Yes | No | Pre | 25 | [here](https://storage.cloud.google.com/aiml-capstone/deeplstm/full_context_withoutstopwords_model_epoch_deeplstm_gpu.h5)
Without stopwords | Bi-LSTM + GloVe | No | No | Pre | 10 | [here](https://storage.cloud.google.com/aiml-capstone/bilstm/full_context_withoutstopwords_model_epoch_10_bilstm_cpu.h5)
Without stopwords | Bi-LSTM + GloVe + Q2C Attention | Yes | No | Pre | 25 | [here](https://storage.cloud.google.com/aiml-capstone/bilstm-q2c-attention-glove/full_context_withoutstopwords_nomask_epoch_25_bilstm_q2c-attention_glove_gpu.h5)
Without stopwords | Bi-LSTM + GloVe + Q2C-C2Q Attention | Yes | No | Pre | 25 | [here](https://storage.cloud.google.com/aiml-capstone/bilstm-bidaf-glove/full_context_withoutstopwords_model_epoch_25_bilstm_bidaf_glove_nomask_gpu.h5)
--- | --- | --- | --- | --- | --- | --- | 
With Stopwords | Bi-LSTM + Universal Sentence Encoder + Q2C Attention
With Stopwords | Bi-LSTM + Universal Sentence Encoder + Q2C-C2Q Attention
With Stopwords | BERT + Universal Sentence Encoder 
With Stopwords | GPT2

In [4]:
list_of_models = {
    "lstm-baseline": {
        "name":"LSTM Baseline",
        "loc":"https://storage.cloud.google.com/aiml-capstone/lstmbaseline-0/context_withoutstopwords_model_epoch_24.h5",
    },
    "lstm-glove": {
        "name":"LSTM + GloVe",
        "loc":"https://storage.cloud.google.com/aiml-capstone/lstmbaseline-0/full_context_withoutstopwords_model_epoch_lstmbaseline0_glove_nomask_gpu.h5"
    },
    "deeplstm-glove":{
        "name":"Deep LSTM + GloVe",
        "loc":"https://storage.cloud.google.com/aiml-capstone/deeplstm/full_context_withoutstopwords_model_epoch_deeplstm_gpu.h5"
    },
    "bilstm-glove":{
        "name":"Bi-LSTM + GloVe",
        "loc":"https://storage.cloud.google.com/aiml-capstone/bilstm/full_context_withoutstopwords_model_epoch_10_bilstm_cpu.h5"
    },
    "bilstm-glove-q2c-attention": {
        "name":"Bi-LSTM + GloVe + Q2C Attention",
        "loc":"https://storage.cloud.google.com/aiml-capstone/bilstm-q2c-attention-glove/full_context_withoutstopwords_nomask_epoch_25_bilstm_q2c-attention_glove_gpu.h5"        
    },
    "bilstm-glove-q2c-c2q-attention":{
        "name":"Bi-LSTM + GloVe + Q2C-C2Q Attention",
        "loc":"https://storage.cloud.google.com/aiml-capstone/bilstm-bidaf-glove/full_context_withoutstopwords_model_epoch_25_bilstm_bidaf_glove_nomask_gpu.h5"
    },
    "bilstm-use-q2c-attention":{
        "name":"Bi-LSTM + Universal Sentence Encoder + Q2C Attention",
        "loc":""
    },
    "bilstm-use-q2c-c2q-attention":{
        "name":"Bi-LSTM + Universal Sentence Encoder + Q2C-C2Q Attention",
        "loc":""
    },
    "bert-use":{
        "name":"BERT + Universal Sentence Encoder",
        "loc":""
    },
    "gpt":{
        "name":"GPT",
        "loc":""
    }    
}

list_of_models

{'bert-use': {'loc': '', 'name': 'BERT + Universal Sentence Encoder'},
 'bilstm-glove': {'loc': 'https://storage.cloud.google.com/aiml-capstone/bilstm/full_context_withoutstopwords_model_epoch_10_bilstm_cpu.h5',
  'name': 'Bi-LSTM + GloVe'},
 'bilstm-glove-q2c-attention': {'loc': 'https://storage.cloud.google.com/aiml-capstone/bilstm-q2c-attention-glove/full_context_withoutstopwords_nomask_epoch_25_bilstm_q2c-attention_glove_gpu.h5',
  'name': 'Bi-LSTM + GloVe + Q2C Attention'},
 'bilstm-glove-q2c-c2q-attention': {'loc': 'https://storage.cloud.google.com/aiml-capstone/bilstm-bidaf-glove/full_context_withoutstopwords_model_epoch_25_bilstm_bidaf_glove_nomask_gpu.h5',
  'name': 'Bi-LSTM + GloVe + Q2C-C2Q Attention'},
 'bilstm-use-q2c-attention': {'loc': '',
  'name': 'Bi-LSTM + Universal Sentence Encoder + Q2C Attention'},
 'bilstm-use-q2c-c2q-attention': {'loc': '',
  'name': 'Bi-LSTM + Universal Sentence Encoder + Q2C-C2Q Attention'},
 'deeplstm-glove': {'loc': 'https://storage.cloud.go

# Evaluation Metrics

## bAbi 

Reference Paper = https://arxiv.org/pdf/1502.05698.pdf 

GitHub - https://github.com/facebookarchive/bAbI-tasks

![alt text](https://storage.cloud.google.com/aiml-capstone/Screenshot%202020-06-20%20at%2011.11.50%20AM.png)
![alt text](https://storage.cloud.google.com/aiml-capstone/Screenshot%202020-06-20%20at%2011.11.03%20AM.png)

## SQuAD test dataset and published dev set

Refer to this eval metrics - https://worksheets.codalab.org/rest/bundles/0x6b567e1cf2e041ec80d7098f031c5c9e/contents/blob/

Eval dataset from SQuAD - https://rajpurkar.github.io/SQuAD-explorer/dataset/dev-v2.0.json

Test Dataset from training = 

## News Domain Specific Evaluations

Test on Sample News Context, Question and Answer pairs ??

## 

# Common Functions

## Custom function for preprocessing of context and question

In [ ]:
# remove unwanted chars
# convert to lowercase
# remove unwanted spaces
# remove stop words
stop_words = set(stopwords.words('english')) 

## reference 
def decontracted(phrase):
    """
    This function remooves punctuation from given sentence.
    """

    if(phrase is np.nan):
      return 'impossible'      

    try:      
      # specific
      phrase = re.sub(r"won\'t", "will not", phrase)
      phrase = re.sub(r"can\'t", "can not", phrase)

      # general
      phrase = re.sub(r"n\'t", " not", phrase)
      phrase = re.sub(r"\'re", " are", phrase)
      phrase = re.sub(r"\'s", " is", phrase)
      phrase = re.sub(r"\'d", " would", phrase)
      phrase = re.sub(r"\'ll", " will", phrase)
      phrase = re.sub(r"\'t", " not", phrase)
      phrase = re.sub(r"\'ve", " have", phrase)
      phrase = re.sub(r"\'m", " am", phrase)
      
      # string operation
      phrase = phrase.replace('\\r', ' ')
      phrase = phrase.replace('\\"', ' ')
      phrase = phrase.replace('\\n', ' ')

      phrase = re.sub('[^A-Za-z0-9]+', ' ', phrase.lower())
    except:
      print(phrase)  
    
    return phrase

def preprocess_text(corpus, text_lower_case=True, 
                      special_char_removal=True, stopword_removal=True, remove_digits=False):    
    normalized_text = []
    # normalize each document in the corpus
    for doc in corpus:
        # doc = decontracted(doc)
        # lowercase the text    
        if text_lower_case:
            doc = doc.lower()
        # remove special characters and\or digits    
        if special_char_removal:
            # insert spaces between special characters to isolate them    
            special_char_pattern = re.compile(r'([{.(-)!}])')
            doc = special_char_pattern.sub(" \\1 ", doc)
            doc = remove_special_characters(doc, remove_digits=remove_digits) 

        if stopword_removal:
            doc = remove_stopwords(doc)

        normalized_text.append(doc)
        
    return normalized_text

def remove_special_characters(text, remove_digits=False):
    #Using regex
    pattern = r'[^a-zA-z0-9\s]' if not remove_digits else r'[^a-zA-z\s]'
    text = re.sub(pattern, '', text)
    return text

def remove_stopwords(text):  
    word_tokens = word_tokenize(text) 
    filtered_sentence = [w for w in word_tokens if not w in stop_words]   
    filtered_sentence = [] 
    for w in word_tokens: 
        if w not in stop_words: 
            filtered_sentence.append(w)                 
    return ' '.join(filtered_sentence)

## Answer Span from Context and Answer, and reverse for predicted spans

In [ ]:
def tokenize(sentence):
    """
    Returns tokenised words.
    """
    return nltk.word_tokenize(sentence)

def answer_span(context,ans):
    """
    This funtion returns anwer span start index and end index.
    """
    ans_token = tokenize(ans)
    con_token = tokenize(context)
    ans_len = len(ans_token)
    
    if ans_len!=0 and ans_token[0] in con_token:
    
        indices = [i for i, x in enumerate(con_token) if x == ans_token[0]]        
        try:

            if(len(indices)>1):
                start = [i for i in indices if (con_token[i:i+ans_len] == ans_token) ]
                end = start[0] + ans_len - 1
                return start[0],end

            else:
                start = con_token.index(ans_token[0])
                end = start + ans_len - 1
                return start,end
        except:
            return -1,-1
    else:
        return -1,-1

def span_to_answer(span, context):
  con_token = tokenize(context)  
  return ' '.join(con_token[span[0]:span[1]+1])

## Update and persist params

In [ ]:
### SAVE PARAMS
# Writing to sample.json 

def updateparams():
  with open(model_path + "params.json", "w") as p: 
    p.write(json.dumps(params))
  print("params.jsop updated and can be found in ", model_path + "params.json")  

# updateparams()

In [ ]:
def showparams():
  pprint.pprint(params)

In [ ]:
def loadparams():
  

## Create a common function to generate sequences (useful in prediction)

In [ ]:
# function to generate sequences withg appropiate padding
def generate_question_context_sequence(context, question):
  question_seq = tokenizer.texts_to_sequences(question)
  context_seq = tokenizer.texts_to_sequences(context)
  question_seq = preprocessing.sequence.pad_sequences(question_seq,
                                                      maxlen=params['question_max_length'],
                                                      padding=params['question_pad_seq'])
  context_seq = preprocessing.sequence.pad_sequences(context_seq,
                                                     maxlen=params['context_max_length'],
                                                     padding=params['question_pad_seq'])
  return context_seq, question_seq

## Create a common function to predict and test

In [ ]:
def predit_test(context, question):
  # get sequence for context and question
  c_ = preprocess_text(context)
  q_ = preprocess_text(question,stopword_removal=False)
  c,q = generate_question_context_sequence(c_, q_)  
  y_ = model.predict([q,c])    
  # # for i in range(26062):
  s = np.argmax(y_[0,:params['context_max_length']])
  e = np.argmax(y_[0,params['context_max_length']:])
  answer = span_to_answer((s,e),c_[0])
  
  # print(c.shape,q.shape,y_.shape,s,e,answer)  
  # print(s, e)
  return c_,q_,[s,e],y_,answer

## Load Tokenizer

In [ ]:
def load_tokenizer():
  with open(model_path + "tokenizer.pkl","rb") as infile:
      tokenizer = pickle.load(infile)

len(tokenizer.word_index)

100850

## Load Test Data with Ground Truth Known

In [ ]:
from sklearn.utils import resample,shuffle

def load_data_withoutstopwords():
  #### NOTE THE 2 data frames's
  df_nostopwords = 'test_squad_data_final_context_withoutstopwords.csv'
  # df_withstopwords = 'squad_data_final_withstopword_withpunctuation.csv'
  test_squad_df = pd.read_csv(project_path+df_nostopwords)
  test_squad_df.drop('Unnamed: 0',axis=1,inplace=True)


  test_squad_df["answer_word_span"] = test_squad_df["answer_word_span"].apply(lambda x :eval(x))
  print(test_squad_df.info())
  return test_squad_df

In [35]:
def logits_loss(y_true,logits):
    """
    Custom loss function which minimises log_loss.
    Referance https://stackoverflow.com/questions/50063613/add-loss-function-in-keras
    """
    
    #y_true = tf.cast(y_true,dtype=tf.int32)
    #logits = tf.cast(logits,dtype=tf.float32)
    
    # breaking the tensor into two half's to get start and end label.
    start_label = y_true[:,:params['context_max_length']]
    end_label = y_true[:,params['context_max_length']:]
    
    # braking the logits tensor into start and end part for loss calcultion.
    start_logit = logits[:,:params['context_max_length']]
    end_logit = logits[:,params['context_max_length']:]
    
    start_loss = tf.keras.backend.categorical_crossentropy(start_label,start_logit)
    end_loss = tf.keras.backend.categorical_crossentropy(end_label,end_logit)
    
#     start_loss = tf.losses.sparse_softmax_cross_entropy(labels=start_label, logits=start_logit)
#     end_loss = tf.losses.sparse_softmax_cross_entropy(labels=end_label, logits=end_logit)
    
    # as per paer
    
    loss = start_loss + end_loss
    
    return loss

## Load Model

In [9]:
tf.__version__

'2.2.0'

In [38]:
from tensorflow import keras
def load_mrc_model(model_name):
  model_path = list_of_models[model_name]['loc'].replace('https://storage.cloud.google.com/aiml-capstone/','gbucket/')
  print(model_path)
  # tf.keras.utils.get_file
  # json_file = open('/content/drive/My Drive/AIML-MRC-Capstone/models/lstmbaseline-0/lstmbaseline-model.json', 'r')
  # loaded_model_json = json_file.read()
  # json_file.close()
  # new_model= keras.models.model_from_json(loaded_model_json)
  custom_objects = {"logits_loss": logits_loss}
  new_model = keras.models.load_model('/content/drive/My Drive/AIML-MRC-Capstone/models/lstmbaseline-0/tf-serve',
                                      custom_objects=custom_objects)
  # # # Check its architecture
  new_model.summary()  

In [39]:
load_mrc_model('lstm-baseline')

# list_of_models['bilstm-glove-q2c-c2q-attention']['loc'].replace('https://storage.cloud.google.com/aiml-capstone/','gbucket/')

gbucket/lstmbaseline-0/context_withoutstopwords_model_epoch_24.h5
Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
QUESTION_INPUT (InputLayer)     [(None, 40)]         0                                            
__________________________________________________________________________________________________
QUESTION_EMBEDDING (Embedding)  (None, 40, 100)      10085100    QUESTION_INPUT[0][0]             
__________________________________________________________________________________________________
CONTEXT_INPUT (InputLayer)      [(None, 426)]        0                                            
__________________________________________________________________________________________________
QUESTION_LSTM (LSTM)            (None, 256)          365568      QUESTION_EMBEDDING[0][0]         
__________________________

# Load Existing Models